In [ ]:
from pathlib import Path
import os
from juliacall import Main as jl
import matplotlib.pyplot as plt
import numpy as np
from mhkit import wave
import pandas as pd

In [ ]:
# cd to WEC-DECIDER
os.chdir('./../..')

In [ ]:

maine = (42.583, -69.882)
n_carolina = (34.339, -76.632)
oregon = (43.682, -124.980)
hawaii = (20.515, -158.476)
locs = [maine, n_carolina, oregon, hawaii]

wave_years = [2008, 2009, 2010]
carbon_constraints = [0, 1]
wns = [0.8, 1, 1.2]
zetas = [0.1, 0.3, 0.5]
forces = [1e5, 1e6, 1e7]

In [ ]:
def run_sim(loc,wave_year,carb,wn,zeta,force):
    wave_resource = wave_data_from_mhkit(loc, wave_year)
    CW = capture_width_from_dynamics_controls(wn, zeta, force)
    power_matrix = wave_resource * CW # use mhkit here
    wave_built = run_GenX(power_matrix, carb)

def wave_data_from_mhkit(lat_lon, wave_year):
    data_type = '1-hour'
    parameter = 'omni-directional_wave_power'
    data, metadata = wave.io.hindcast.hindcast.request_wpto_point_data(data_type,parameter,lat_lon,wave_year)
    data_mod = data
    data_mod["Avail"] = data["omni-directional_wave_power_0"] / np.max(data["omni-directional_wave_power_0"]) 
    P_lim_list = np.concatenate( (np.linspace(0.005, 0.025, num=5), np.linspace(0.03, 0.09, num=7), np.linspace(0.1,0.9,num=9)) )

    for P_limit in P_lim_list:
        key = "Avail_limited_"+str(int(P_limit*1000))
        data_mod[key] = np.minimum(data_mod["Avail"], P_limit) / P_limit

    means = data_mod.mean()
    CFs = np.array(means)[1:]
    print(means)
    CFs = np.concatenate([CFs[1:],CFs[:1]])
    print(CFs)
    data_mod.head()

def capture_width_from_dynamics_controls(wn, zeta, force):
    pass

def run_GenX(power_matrix, carb):
    jl.seval("""cd("modules/CEM")""")
    jl.seval("""Pkg.instantiate()""")
    jl.seval("""Pkg.status()""")
    jl.seval("""Pkg.project()""")
    
    jl.seval("""include("caserunner.jl")""")
    jl.seval("""run_caserunner(" """ + str(pg_id) + """ ", " """ + loc + """ ")""")
    pg_id += 1

In [ ]:
for loc in locs:
    for wave_year in wave_years:
        for carb in carbon_constraints:
            for wn in wns:
                for zeta in zetas:
                    for force in forces:
                        run_sim(loc,wave_year,carb,wn,zeta,force)